# LFM梯度下降算法实现

### 0.引入依赖

In [1]:
import numpy as np
import pandas as pd

### 1.数据准备

In [2]:
#评分矩阵
R = np.array([[4,0,2,0,1],
             [0,2,3,0,0],
             [1,0,2,4,0],
             [5,0,0,3,1],
             [0,0,1,5,1],
             [0,3,2,4,1],])
len(R[0])

5

### 2.算法实现

In [20]:
'''
@输入参数：
R：M*N的评分矩阵
K：隐特征向量维度
max_iter：最大迭代次数
alpha：步长
lamda：正则化系数

@输出
分解之后的P，Q
P：初始化用户特征矩阵M*K
Q：初始化物品特征矩阵N*K
'''

#给定超参数

K = 5
max_iter = 5000
alpha = 0.0002
lamda = 0.004

#核心算法
def LFM_grad_desc(R,K=2,max_iter=1000,alpha=0.0001,lamda=0.002):
    #基本维度参数
    M = len(R)
    N = len(R[0])
    
    #P,Q初始化值，随机生成
    P = np.random.rand(M,K)
    Q = np.random.rand(N,K)
    Q = Q.T
    
    #开始迭代
    for step in range(max_iter):
        #所有用户u、物品i做遍历，对应的特征向量Pu、Qi梯度下降
        for u in range(M):
            for i in range(N):
                #对于每一个大于0的评分，求出预测评分误差
                if R[u][i] > 0:
                    eui = np.dot( P[u,:], Q[:,i] ) - R[u][i]
                    
                    #代入公式，按照梯度下降算法更新当前Pu，Qi
                    for k in range(K):
                        P[u][k] = P[u][k] - alpha*(2*eui*Q[k][i]+2*alpha*P[u][k])
                        Q[k][i] = Q[k][i] - alpha*(2*eui*P[u][k]+2*alpha*Q[k][i])
        
        #u、i遍历完成，所有特征向量更新完成，可以得到P，Q，可以计算预测评分矩阵
        predR = np.dot(P,Q)
        
        #计算当前损失函数
        cost = 0
        for u in range(M):
            for i in range(N):
                if R[u][i] > 0:
                    cost += (np.dot(P[u,:],Q[:,i]) - R[u][i])**2
                    #加上正则化项
                    for k in range(K):
                        cost += lamda*(P[u][k]**2+Q[k][i]**2)
        if cost<0.0001:
            break
        
    
    return P,Q.T,cost

### 3.测试

In [22]:
P, Q, cost = LFM_grad_desc(R,K,max_iter,alpha,lamda)

print(P)
print(Q)
print(cost)

predR = P.dot(Q.T)
print(R)
predR

[[ 0.49454339  0.64047227  0.51952499  1.16352289  0.86304857]
 [ 0.4310208   0.17368038  1.11090158  1.36471766  0.08325639]
 [-0.76688075  0.54274155  1.28962412  0.76849973  0.50872009]
 [ 1.0752874   0.66021323  0.19628833  0.84433564  1.10915769]
 [ 0.39475392  1.57454612  0.83584495  0.25815515  1.10498781]
 [ 0.71834042  0.82841992  1.0601268   0.62460502  0.62439114]]
[[ 1.76620963  1.08359101  0.24557437  0.99577524  1.3422584 ]
 [ 1.0704778   0.5719686   0.96954507  0.22020869  0.91317201]
 [ 0.30187861 -0.14591915  1.07884458  1.24632047 -0.07668373]
 [-0.00812173  1.47130679  1.65027484  0.7372873   0.98334093]
 [ 0.20522059  0.08273326  0.31612851  0.16333253  0.49481169]]
0.5447106242268768
[[4 0 2 0 1]
 [0 2 3 0 0]
 [1 0 2 4 0]
 [5 0 0 3 1]
 [0 0 1 5 1]
 [0 3 2 4 1]]


array([[4.01210079, 2.44376028, 2.00026224, 3.50219531, 0.93580316],
       [2.69298418, 2.01435715, 2.99775396, 3.17338782, 0.71811017],
       [0.99842313, 1.37363027, 1.9993891 , 3.99985234, 0.67245166],
       [4.99233223, 2.91778471, 1.40729156, 2.99977082, 1.02407671],
       [4.34888795, 3.19944753, 1.02816758, 4.96972245, 1.0644394 ],
       [3.88680858, 2.97835833, 1.97025872, 4.037029  , 0.96206679]])